# liberaries init

In [1]:
import mne
import numpy as np
from pathlib import Path
from typing import Dict, List, Tuple
import shutil

import os

import pandas as pd
from pandas import DataFrame
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import mne
import torch
from torch.utils.data import DataLoader ,  Dataset
from torch import Tensor
from torch import nn
from torch.optim.lr_scheduler import ExponentialLR ,    MultiStepLR
import torch.nn.functional as F




In [2]:
#global use

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

# Data preproccessing

## preparing the data 

In [3]:
def load_participant_files_from_dir(dir_path: Path) -> dict[str ,  List[Path] ]:
    """
    from participants directory returns files path orginized in relation to task and run 
    exemple 
    dic = {
    "contrastchangeDetection_run-1" : [path1,path2,path3],
    ...   
    }

    """
    dir_path = os.path.join(dir_path , "eeg")
    participant_files = {}
    for file in os.listdir(dir_path):
        elements = file.split("_")
        if len(elements ) == 3 :
            elements.insert(2 , "run-1")
        elements[1] += "_"+ elements[2]
        del elements[2]
        if elements[1] not in participant_files:
            participant_files[elements[1]] = [Path(os.path.join(dir_path , file))]
        else:
            participant_files[elements[1]].append(Path(os.path.join(dir_path , file)))
    # aka each task has the events , channels , eeg json and eeg raw
    for key in participant_files  :
        assert len(participant_files[key]) == 4 
    return participant_files
def prepare_ccd_events(events_fp : Path) -> DataFrame:
    """
    from events file returns a dataframe with trial start , trial end , stimulas start , action onset , RT AND SUCCESS

    """
    assert os.path.splitext(events_fp)[1] == ".tsv"
    events = pd.read_csv(events_fp , sep = "\t")
    events["onset"] = pd.to_numeric(events["onset"],errors="raise")   
    events = events.reset_index(drop=True)
    events = events.sort_values(by="onset" , ascending=True)
    trials = events[ events["value"] == "contrastTrial_start"].copy()
    trials["trial_start"] = trials["onset"]

    trials["trial_end"] = trials["onset"].shift(-1) 
    stimulas = events [ events["value"].isin(["right_target" ,"left_target"])].copy()
    action = events [ events["value"].isin(["right_buttonPress" ,"left_buttonPress"])].copy()
    results = []
    for i in range(0 ,len(trials)-1 ):
        #get the stimulas onset in the trial i duration
        stimulas_row = stimulas[ (stimulas["onset"] >= trials["trial_start"].iloc[i]) & (stimulas["onset"] < trials["trial_end"].iloc[i]) ]
        if stimulas_row.empty:
            continue
        stimulas_start = float(stimulas_row["onset"].iloc[0])

        action_rows = action[ (action["onset"] >= stimulas_start) & (action["onset"] < trials["trial_end"].iloc[i]) ]
        # if theres no action , theres no rt , theres no success
        if action_rows.empty:
            
            continue
        action_row = action_rows.iloc[0]
        action_onset = float(action_row["onset"])
        rt = action_onset - stimulas_start
        success = 1 if action_row["feedback"] == "smiley_face" else 0
        result ={
        "trial_start" : float(trials["trial_start"].iloc[i]) ,
        "trial_end" :float(trials["trial_end"].iloc[i]) ,
        "stimulas_start" : stimulas_start,
        "action_onset" :action_onset  ,
        "rt" : rt ,
        "success" : success
        }
        results.append(result)
    return pd.DataFrame(results)


    

def prepare_participants_ccd_data(data_dir: Path) -> Dict[str , Dict[str , Tuple[DataFrame , Path]]]:

    #dictionary that will have for each  participant (path as key) a dictionary with the ccd-run (key) and values as the df , and path to raw eeg file 
    results = {}
    for release in os.listdir(data_dir):
        release_dir_path = os.path.join(data_dir , release)
        #go throught the participants directory
        for file in os.listdir(release_dir_path):
            
            if not  file.split("-")[0] == "sub" :
                continue

            participant_id = file
            participant_dir_path = os.path.join(release_dir_path , file)

            results[participant_dir_path] = {}
            participant_files = load_participant_files_from_dir(participant_dir_path)
            filtered_participant_files = {}
            # filter for ccd and sus data
            for key in participant_files:
                if key.split("_")[0].lower() == "task-contrastchangedetection" :
                    filtered_participant_files[key] = participant_files[key]
            
            for task , files in filtered_participant_files.items():
                events_path = [path for path in files if "events" in str(path)]  
                assert len(events_path) == 1
                events_path = events_path[0]
                eeg_path = [path for path in files if ".set" in str(path)]

                assert len(eeg_path) == 1
                df = prepare_ccd_events(events_path)
                results[participant_dir_path][task] = (df , eeg_path[0])
    return results

def participants_ccd_data_to_list(data : dict) -> List[Tuple[DataFrame , Path]]:
    results = []
    for participant in data:
        for task in data[participant]:
            results.append(data[participant][task])
    return results
            
def participants_ccd_list_to_trial_rt_pair(data : List[Tuple[DataFrame , Path]]) -> List[Tuple[Path ,Tuple[float,float] , float]]:
    results = []
    for participant in data:
        df , eeg_path = participant
        for i in range(0 , len(df)):
            results.append((eeg_path , (df["stimulas_start"].iloc[i]+0.5 , df["stimulas_start"].iloc[i]+2.5 ) , df["rt"].iloc[i]))
    return results

def train_val_test_split_by_subject(data : List[Tuple[Path ,Tuple[float,float] , float]] , test_size : float = 0.1 , val_size : float = 0.1) -> Tuple[List[Tuple[Path ,Tuple[float,float] , float]] , List[Tuple[Path ,Tuple[float,float] , float]] , List[Tuple[Path ,Tuple[float,float] , float]]]:
    subjects = []
    for element in data:
        path = element[0]
        subject = str(path).split('\\')[-3]
        if subject not in subjects:
            subjects.append(subject)
    train_subjects , test_subjects = train_test_split(subjects , test_size=test_size +val_size)
    test_subjects , val_subjects = train_test_split(test_subjects , test_size=val_size/(test_size +val_size))
    train_data = []
    test_data = []
    val_data = []
    for element in data:
        path = element[0]
        subject = str(path).split('\\')[-3]
        if subject in train_subjects:
            train_data.append(element)
        elif subject in test_subjects:
            test_data.append(element)
        elif subject in val_subjects:
            val_data.append(element)
    return train_data , val_data , test_data



In [4]:
data_path= r"C:\disque d\ai_stuff\projects\pytorchtraining\eeg_competition\data"
data = prepare_participants_ccd_data(data_path)

data = participants_ccd_data_to_list(data)
data = participants_ccd_list_to_trial_rt_pair(data)
train , val , test = train_val_test_split_by_subject(data)
print(len(data))

10500


## extracting the raw eeg windows data and setting them up for fast import

In [5]:
def pairs_to_fast_loading_shards(data : List[Tuple[Path ,Tuple[float,float] , float]] , split = "train" , shard_size : int = 1000):
    shards_path = "shards_dir"
    split_path = os.path.join(shards_path , split)

    if not os.path.exists(shards_path):
        os.makedirs(shards_path)

    if not os.path.exists(split_path):
        os.makedirs(split_path)
    else:
        return split_path


    shard_index=0
    window_shard_path = os.path.join(split_path , f"window_shard_{shard_index}.npy")
    rt_shard_path = os.path.join(split_path , f"rt_shard_{shard_index}.npy")
    windows =[]
    rts =[]
    for index , element in tqdm(enumerate(data)):
        if index!=0 and index % shard_size == 0:
            window_array = np.array(windows)
            rt_array = np.array(rts)
            X = np.lib.format.open_memmap(window_shard_path , dtype='float32' , mode='w+' , shape=(window_array.shape[0] , window_array.shape[1] , window_array.shape[2]))
            X[:] = window_array
            del X
            Y = np.lib.format.open_memmap(rt_shard_path , dtype='float32' , mode='w+' , shape=(rt_array.shape[0] ,))
            Y[:] = rt_array
            del Y
            shard_index+=1
            window_shard_path = os.path.join(split_path , f"window_shard_{shard_index}.npy")
            rt_shard_path = os.path.join(split_path , f"rt_shard_{shard_index}.npy")
            windows =[]
            rts =[]
        eeg_path , (start , end) , rt = element
        eeg = mne.io.read_raw_eeglab(eeg_path , preload=True)
        eeg = eeg.crop(start , end+0.2)
        raw = eeg.get_data()[: ,:200]
        windows.append(raw)
        rts.append(rt)
    window_array = np.array(windows)
    rt_array = np.array(rts)
    if len(window_array) > 0:
        X = np.lib.format.open_memmap(window_shard_path , dtype='float32' , mode='w+' , shape=(window_array.shape[0] , window_array.shape[1] , window_array.shape[2]))
        X[:] = window_array
        del X
        Y = np.lib.format.open_memmap(rt_shard_path , dtype='float32' , mode='w+' , shape=(rt_array.shape[0] ,))
        Y[:] = rt_array
        del Y
    return split_path
    


In [6]:
shard_size = 1000
pairs_to_fast_loading_shards(train , "train" ,shard_size)
pairs_to_fast_loading_shards(test , "test",shard_size)
pairs_to_fast_loading_shards(val , "val",shard_size)

8478it [15:57,  8.85it/s]
1108it [01:52,  9.86it/s]
914it [01:31, 10.01it/s]


'shards_dir\\val'

In [7]:
class EEGDataset(Dataset):
    def __init__(self , shards_path , shard_size  =1000):
        self.shards_path = shards_path
        self.numbers_of_shards = int( len(os.listdir(self.shards_path)) / 2)
        self.shard_size = shard_size
        
        
    def __len__(self):
        length = 0
        for shard_path in os.listdir(self.shards_path):
            if "window" in shard_path:
                window_shard_path = os.path.join(self.shards_path , shard_path)
                shard = np.lib.format.open_memmap(window_shard_path , mode="r")
                size = shard.shape[0]
                length += size
        return length
    
    def __getitem__(self , index):
        shard_pos = index // self.shard_size 
        window_shard_path = os.path.join(self.shards_path , f"window_shard_{shard_pos}.npy")
        X = np.lib.format.open_memmap(window_shard_path , mode="r")
        rt_shard_path = os.path.join(self.shards_path , f"rt_shard_{shard_pos}.npy")
        Y = np.lib.format.open_memmap(rt_shard_path , mode="r")
        raw = X[index % self.shard_size]
        rt = Y[index % self.shard_size]
        return raw , rt
    


        


In [8]:
train_pairs_shards_path = pairs_to_fast_loading_shards(train , "train" , shard_size)
test_pairs_shards_path = pairs_to_fast_loading_shards(test , "test" ,  shard_size)
val_pairs_shards_path =pairs_to_fast_loading_shards(val , "val" , shard_size)
train_data = EEGDataset(train_pairs_shards_path ,  shard_size)
test_data = EEGDataset(test_pairs_shards_path , shard_size)
val_data = EEGDataset(val_pairs_shards_path ,shard_size)
eeg , rt=train_data[0]
print(eeg.shape)
print(rt)


(129, 200)
1.768


# Baseline model training

In [107]:
def nrmse_over_data(model, dataloader, device):
    model.eval()
    se_sum = 0.0     # sum of squared errors
    sum_y = 0.0      # sum of y
    sum_y2 = 0.0     # sum of y^2
    n = 0

    with torch.inference_mode():
        for x, y in dataloader:
            x = x.to(device, dtype=torch.float32)
            y = y.to(device, dtype=torch.float32)

            y_pred = model(x).view_as(y)
            diff = y_pred - y

            se_sum += diff.pow(2).sum().item()
            sum_y  += y.sum().item()
            sum_y2 += y.pow(2).sum().item()
            n += y.numel()

    rmse = (se_sum / n) ** 0.5
    var  = (sum_y2 / n) - (sum_y / n) ** 2
    std  = var ** 0.5
    return rmse / std


In [116]:
class Importance_extraction(nn.Module):
    def __init__(self , nb_channels = 129 , nb_times= 200 ):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear( nb_times , 256),
            nn.ReLU(),
            nn.Linear(256 , 512),
            nn.ReLU(),
            nn.Linear(512 , 512),
            nn.ReLU(),
            nn.Linear(512 , 256),
            nn.ReLU(),
            nn.Linear(256 , 64),
            nn.ReLU(),
            nn.Linear(64 , 1)   
        )
        self.softmax = nn.Softmax(dim=1)
    def forward(self , x):
        x = self.classifier(x)
        return self.softmax(x)  
      
    
class Section_attention(nn.Module):
    def __init__(self , nb_channels=129 , nb_times=10):
        super().__init__()
        self.nb_channels = nb_channels
        self.nb_times = nb_times
        self.softmax = nn.Softmax(dim=-1)

    def forward(self , Q , K , V):
        attn = self.softmax((Q @ K.transpose(-1 , -2)) / (self.nb_times ** 0.5))
        res = attn @ V
        return res


class Multi_section_attention(nn.Module):
    def __init__(self , nb_channels=129 , nb_times=200 , nb_output=1 , section_size=10):
        super().__init__()
        self.importance  = Importance_extraction(nb_channels , nb_times)
        self.section_attention = Section_attention(nb_channels , section_size)

        self.section_size = section_size
        self.nb_of_sections = nb_times // section_size
        self.ll = nn.Linear(nb_times , nb_times)
        self.nb_times = nb_times
        self.nb_channels = nb_channels

        self.to_q = nn.Linear(nb_times , nb_times)
        self.to_k = nn.Linear(nb_times , nb_times)
        self.to_v = nn.Linear(nb_times , nb_times)

    def forward(self , x):
        imp = self.importance(x)
        adjusted_x = x * imp

        Q = self.to_q(adjusted_x)
        K = self.to_k(adjusted_x)
        V = self.to_v(adjusted_x)

        Q = Q.reshape(
            adjusted_x.shape[0] , adjusted_x.shape[1] , self.nb_of_sections , self.section_size
        ).transpose(1 , 2)  # (b, sections, c, section_size)
        
        K = K.reshape(
            adjusted_x.shape[0] , adjusted_x.shape[1] , self.nb_of_sections , self.section_size
        ).transpose(1 , 2)  # (b, sections, c, section_size)
        V =V.reshape(
            adjusted_x.shape[0] , adjusted_x.shape[1] , self.nb_of_sections , self.section_size
        ).transpose(1 , 2)  # (b, sections, c, section_size)
        

        x = self.section_attention(Q , K , V)
        x = x.transpose(1 , 2).reshape(x.shape[0] , self.nb_channels , self.nb_times)

        return x


class LayerNorm(nn.Module):
    def __init__(self ,  nb_times ):
        super().__init__()
        self.gamma = nn.Parameter(torch.ones(nb_times,) ,  requires_grad=True)
        self.beta = nn.Parameter(torch.zeros(nb_times,) ,  requires_grad=True)
    def forward(self , original  , attention_res):
        add = original + attention_res
        mean = add.mean(dim=-1,keepdim=True)
        stand_div = add.std(dim=-1 , keepdim=True) + 1e-4 # just in case el std ta7 lel 0
        result = (add - mean)/stand_div
        return self.gamma * result + self.beta

class BaseLineModel(nn.Module):
    def __init__(self , nb_channels = 129 , nb_times= 200 , nb_output = 1 , section_size = 20):
        super().__init__()
        self.multi_section_attention = Multi_section_attention(nb_channels , nb_times , nb_output , section_size)
        self.add_norm = LayerNorm(nb_times)
        self.classifer = nn.Sequential(
            nn.Flatten(),
            nn.Linear(nb_times * nb_channels , 256),
            nn.ReLU(),
            nn.Linear(256 , 512),
            nn.ReLU(),
            nn.Linear(512 , 512),
            nn.ReLU(),
            nn.Linear(512 , 256),
            nn.ReLU(),
            nn.Linear(256 , 64),
            nn.ReLU(),
            nn.Linear(64 , 1),
        )
    def forward(self , x):
        attention_res = self.multi_section_attention(x)
        x= self.add_norm(x , attention_res)
        x = self.classifer(x)
        return x

### chat enhanced


In [129]:
class Section_attention(nn.Module):
    def __init__(self, nb_channels=129, section_size=10, heads=4, d_k=8):
        super().__init__()
        self.nb_channels = nb_channels
        self.section_size = section_size
        self.heads = heads
        self.d_k = d_k
        self.scale = d_k ** 0.5
        self.softmax = nn.Softmax(dim=-1)
        self.out = nn.Linear(heads * d_k, section_size)  # bring back to section_size

    def forward(self, Q, K, V):
        # Q,K,V: (b, sections, c, heads, d_k)
        # move heads before c for matmul
        Q = Q.permute(0,1,3,2,4)  # (b, sections, heads, c, d_k)
        K = K.permute(0,1,3,2,4)  # (b, sections, heads, c, d_k)
        V = V.permute(0,1,3,2,4)  # (b, sections, heads, c, d_k)

        attn = self.softmax((Q @ K.transpose(-1, -2)) / self.scale)   # (b, sections, heads, c, c)
        x = attn @ V                                                  # (b, sections, heads, c, d_k)

        # merge heads back into feature dim per channel, then project to section_size
        x = x.permute(0,1,3,2,4).contiguous()                         # (b, sections, c, heads, d_k)
        x = x.view(x.shape[0], x.shape[1], x.shape[2], -1)            # (b, sections, c, heads*d_k)
        x = self.out(x)                                               # (b, sections, c, section_size)
        return x


class Multi_section_attention(nn.Module):
    def __init__(self, nb_channels=129, nb_times=200, nb_output=1, section_size=10, heads=4, d_k=8):
        super().__init__()
        self.importance  = Importance_extraction(nb_channels, nb_times)
        self.section_size = section_size
        self.nb_of_sections = nb_times // section_size
        self.nb_times = nb_times
        self.nb_channels = nb_channels
        self.heads = heads
        self.d_k = d_k

        # QKV AFTER sectioning: project last dim (section_size) → heads*d_k
        self.to_q = nn.Linear(section_size, heads * d_k)
        self.to_k = nn.Linear(section_size, heads * d_k)
        self.to_v = nn.Linear(section_size, heads * d_k)

        self.section_attention = Section_attention(nb_channels, section_size, heads, d_k)

    def forward(self, x):
        imp = self.importance(x)                       # (b, c, 1) softmax over channels
        adjusted_x = x * imp                           # (b, c, t)

        # split into sections
        z = adjusted_x.reshape(adjusted_x.shape[0], adjusted_x.shape[1],
                               self.nb_of_sections, self.section_size) \
                        .transpose(1, 2)               # (b, sections, c, section_size)

        # per-section projections
        Q = self.to_q(z).view(z.shape[0], z.shape[1], z.shape[2], self.heads, self.d_k)  # (b, sections, c, h, d_k)
        K = self.to_k(z).view(z.shape[0], z.shape[1], z.shape[2], self.heads, self.d_k)
        V = self.to_v(z).view(z.shape[0], z.shape[1], z.shape[2], self.heads, self.d_k)

        x = self.section_attention(Q, K, V)            # (b, sections, c, section_size)
        x = x.transpose(1, 2).reshape(x.shape[0], self.nb_channels, self.nb_times)  # (b, c, t)
        return x

class BaseLineModel(nn.Module):
    def __init__(self, nb_channels=129, nb_times=200, nb_output=1, section_size=10, heads=4, d_k=8):
        super().__init__()
        self.multi_section_attention = Multi_section_attention(
            nb_channels, nb_times, nb_output, section_size, heads, d_k
        )
        self.add_norm = LayerNorm(nb_times)

        self.classifer = nn.Sequential(
            # pool over time → (b, c)
            nn.Identity(),  # keep style
        )
        self.dropout = nn.Dropout(p=0.2)
        self.fc1 = nn.Linear(nb_channels, 128)
        self.fc2 = nn.Linear(128, 64)
        self.out = nn.Linear(64, 1)

    def forward(self, x):
        att = self.multi_section_attention(x)      # (b, c, t)
        x = self.add_norm(x, att)                  # (b, c, t)

        x = x.mean(dim=-1)                         # (b, c)   <- key change
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x


In [130]:
batch_size = 32

train_dataloader = DataLoader(train_data , batch_size=batch_size , shuffle=True )
test_dataloader = DataLoader(test_data , batch_size=batch_size , shuffle=True )
val_dataloader = DataLoader(val_data , batch_size=batch_size , shuffle=True )

In [131]:
# testing model is working
test_model = BaseLineModel().to(device)

for  batch in tqdm(train_dataloader):
    x , y = batch
    x = x.to(device)
    y = y.to(device)
    y_pred = test_model(x)
    break

  0%|          | 0/265 [00:00<?, ?it/s]

In [132]:

blmodel = BaseLineModel().to(device)

lr = 1e-3

optimizer = torch.optim.Adam(blmodel.parameters() , lr=lr  )
scheduler1 = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer , patience=3 )

loss_f = nn.MSELoss()


In [133]:



print("before training")
print(f"train nRMSE : {nrmse_over_data(blmodel , train_dataloader ,device)}")
print(f"test nRMSE : {nrmse_over_data(blmodel , test_dataloader ,device)}")

epochs = 30
for epoch in range(epochs):
    blmodel.train()
    cumulative_loss = 0
    for  batch in tqdm(train_dataloader):
        x , y = batch
        x = x.to(device)
        y = y.to(device)
        y_pred = blmodel(x)
        loss = loss_f(y_pred.squeeze(-1) , y)
        cumulative_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    nrmse_train = nrmse_over_data(blmodel , train_dataloader ,device)
    if lr!= optimizer.param_groups[0]['lr']:
        print(f"new lr : {optimizer.param_groups[0]['lr']}")
        lr = optimizer.param_groups[0]['lr']

    print(f"train epoch : {epoch +1} , loss : {cumulative_loss/len(train_dataloader)} , nRMSE : {nrmse_train}")
    blmodel.eval()
    with torch.inference_mode():
        cumulative_loss = 0
        for batch in tqdm(test_dataloader):
            x , y = batch
            x = x.to(device)
            y = y.to(device)
            y_pred = blmodel(x)
            loss = loss_f(y_pred.squeeze(-1) , y)
            cumulative_loss += loss.item()
        nrmse_over_test = nrmse_over_data(blmodel , test_dataloader ,device)
        scheduler1.step(nrmse_over_test)
        print(f"test epoch : {epoch +1} , loss : {cumulative_loss/len(test_dataloader)} , nRMSE : {nrmse_over_data(blmodel , test_dataloader,device)}")
    






before training
train nRMSE : 4.480052098182609
test nRMSE : 4.717848996036528


100%|██████████| 265/265 [00:10<00:00, 24.14it/s]


train epoch : 1 , loss : 0.40520924464711605 , nRMSE : 1.0025569050593914


100%|██████████| 35/35 [00:01<00:00, 19.90it/s]


test epoch : 1 , loss : 0.14846871665545872 , nRMSE : 1.0097055700236715


...

test epoch : 29 , loss : 0.14669835290738514 , nRMSE : 1.0031202575620908


100%|██████████| 265/265 [00:08<00:00, 31.33it/s]


train epoch : 30 , loss : 0.16365921197915978 , nRMSE : 1.0000898296403018


100%|██████████| 35/35 [00:01<00:00, 34.83it/s]


test epoch : 30 , loss : 0.14650954497712 , nRMSE : 1.0031195635953858


## final test on r5

In [122]:
path_to_r5 = r"C:\disque d\ai_stuff\projects\pytorchtraining\eeg_competition\final_tests\final_r5_test"
data = prepare_participants_ccd_data(path_to_r5)

data = participants_ccd_data_to_list(data)
data = participants_ccd_list_to_trial_rt_pair(data)

final_r5_test =pairs_to_fast_loading_shards(data ,"final_r5_test" , shard_size)
r5_test_data = EEGDataset(final_r5_test ,  shard_size)


In [134]:
r5_test_dataloader = DataLoader(r5_test_data , batch_size=batch_size , shuffle=False )
print(f"r5 test nRMSE : {nrmse_over_data(blmodel , r5_test_dataloader,device)}")

r5 test nRMSE : 1.0011487058935713


In [136]:
def calculate_rmse(blmodel , dataloader , device):
    blmodel.eval()
    se_sum = 0.0   # sum of squared errors
    n = 0          # total number of elements
    
    with torch.inference_mode():
        for x, y in tqdm(dataloader):
            x = x.to(device)
            y = y.to(device)

            y_pred = blmodel(x).squeeze(-1)
            diff = y_pred - y

            se_sum += (diff ** 2).sum().item()
            n += y.numel()

    rmse = np.sqrt(se_sum / n)
    return rmse

print(f"r5 test RMSE : {calculate_rmse(blmodel , r5_test_dataloader,device)}")

100%|██████████| 38/38 [00:01<00:00, 32.17it/s]

r5 test RMSE : 0.3627938385318386


# full data preproccessing for encoder training

In [2]:
def load_participant_eeg_files_from_dir(dir_path: Path) -> dict[str ,  List[Path] ]:
    """
    from participants directory returns files path orginized in relation to task and run 
    exemple 
    dic = {
    "contrastchangeDetection_run-1" : [path1,path2,path3],
    ...   
    }

    """
    dir_path = os.path.join(dir_path , "eeg")
    participant_files = {}
    for file in os.listdir(dir_path):
        if os.path.splitext(file)[1] != ".set":
            continue
        elements = file.split("_")
        if len(elements ) == 3 :
            elements.insert(2 , "run-1")
        elements[1] += "_"+ elements[2]
        del elements[2]
        participant_files[elements[1]] = Path(os.path.join(dir_path , file))


    return participant_files


    

def prepare_participants_eeg_data(data_dir: Path) -> Dict[str , Dict[str , Tuple[DataFrame , Path]]]:

    #dictionary that will have for each  participant (path as key) a dictionary with the ccd-run (key) and values as the df , and path to raw eeg file 
    results = {}
    for release in os.listdir(data_dir):
        release_dir_path = os.path.join(data_dir , release)
        #go throught the participants directory
        for file in os.listdir(release_dir_path):
            
            if not  file.split("-")[0] == "sub" :
                continue

            participant_id = file
            participant_dir_path = os.path.join(release_dir_path , file)

            participant_files = load_participant_eeg_files_from_dir(participant_dir_path)
            results[participant_dir_path] = [list(participant_files.values())]
        

    return results

def train_val_test_split_by_subject_full_data(data : Dict[str , List[Path]], test_size = 0.1 , val_size = 0.1) -> Tuple[List[Path] , List[Path] , List[Path]]:
    subjects = list(data.keys())
    train , test = train_test_split(subjects , test_size=test_size + val_size  )
    test , val = train_test_split(test , test_size = val_size/(test_size + val_size) )
    train_data_paths =[]
    for subject in train:
        for element in data[subject]:
            train_data_paths.extend(element)
    test_data_paths = []
    for subject in test:
        for element in data[subject]:
            test_data_paths.extend(element)
    val_data_paths = []
    for subject in val:
        for element in data[subject]:
            val_data_paths.extend(element)
    return train_data_paths , test_data_paths , val_data_paths  

def load_and_shard_eeg_data(data_paths: List[Path], split_type ="train" , shard_size = 1000 , window_size = 200 , stride = 20 ,  output_dir = "full_eeg_data_shards")-> Tuple[Path  , int] :
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    shard_path = os.path.join(output_dir , split_type)
    if not os.path.exists(shard_path):
        os.mkdir(shard_path)
    else :
        size =0
        for file in os.listdir( shard_path):
            full_path = os.path.join(shard_path , file)
            X = np.lib.format.open_memmap(full_path , mode="r" , dtype=np.float32)
            size += X.shape[0]
            
        return shard_path , size
    eeg_arrays = []
    current_size = 0
    index = 0
    full_number_of_windows =0
    print(f"proccessing {len(data_paths)} files")
    for i , data_path in tqdm(enumerate(data_paths)):
        eeg_data = mne.io.read_raw_eeglab(data_path, preload=True, verbose=False)
        eeg_data = eeg_data.get_data()
        eeg_data_array = np.array(eeg_data)
        eeg_data_array = eeg_data_array[:, : eeg_data_array.shape[1]//window_size*window_size]
        eeg_data_array = eeg_data_array.reshape(eeg_data_array.shape[0] , eeg_data_array.shape[1]//window_size , window_size).transpose(1 , 0 , 2)
        
        eeg_arrays.append(eeg_data_array)
        current_size += eeg_data_array.shape[0]
        full_number_of_windows += eeg_data_array.shape[0]
        if current_size >= shard_size:
            new_eeg_data_windows = np.concatenate(eeg_arrays , axis=0)
            to_be_added = new_eeg_data_windows[: shard_size]

            if current_size == shard_size:
                eeg_arrays = []
            else :
                eeg_arrays = [new_eeg_data_windows[shard_size:]]
            
            
            X = np.lib.format.open_memmap(os.path.join(shard_path , f"window_shard_{index}.npy") , mode="w+" , dtype=np.float32 , shape=(to_be_added.shape[0] , to_be_added.shape[1] , to_be_added.shape[2]))
            X[:] = to_be_added
            X.flush()
            current_size = 0
            index+=1

    new_eeg_data_windows = np.concatenate(eeg_arrays , axis=0)
    to_be_added = new_eeg_data_windows[: shard_size]

    if current_size == shard_size:
        eeg_arrays = []
    else :
        eeg_arrays = [new_eeg_data_windows[shard_size:]]
            
            
    X = np.lib.format.open_memmap(os.path.join(shard_path , f"window_shard_{index}.npy") , mode="w+" , dtype=np.float32 , shape=(to_be_added.shape[0] , to_be_added.shape[1] , to_be_added.shape[2]))
    index+=1
    X[:] = to_be_added
    X.flush()
    full_number_of_windows += to_be_added.shape[0]

    return shard_path , full_number_of_windows
    

        


In [3]:
path = r"D:\projects\pytorch_training\data"
output_dir = r"D:\projects\pytorch_training\full_eeg_data_shards"
detailed_data = prepare_participants_eeg_data(path)
train_data_paths , test_data_paths , val_data_paths = train_val_test_split_by_subject_full_data(detailed_data)




FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:\\projects\\pytorch_training\\data'

In [ ]:
train_dir_path  , train_windows_number = load_and_shard_eeg_data(train_data_paths , split_type="train" , shard_size=1000 , window_size=200 , stride=20 , output_dir=output_dir)
print(f"train data path  { train_dir_path} with {train_windows_number} windows")
val_dir_path , val_windows_number = load_and_shard_eeg_data(val_data_paths , split_type="val" , shard_size=1000 , window_size=200 , stride=20 , output_dir=output_dir)
test_dir_path , test_windows_number = load_and_shard_eeg_data(test_data_paths , split_type="test" , shard_size=1000 , window_size=200 , stride=20 , output_dir=output_dir)


train data path  D:\projects\pytorch_training\full_eeg_data_shards\train with 218000 windows
proccessing 212 files


212it [02:58,  1.19it/s]


proccessing 211 files


211it [03:14,  1.09it/s]


In [4]:
class Encoder_Dataset(Dataset):
    def __init__(self , data_dir ,real_nb_windows , window_size = 200 , stride = 20 , split = "train" , shard_size = 1000):
        self.data_dir = data_dir
        self.real_nb_windows = real_nb_windows
        self.window_size = window_size
        self.stride = stride
        self.split = split
        self.shard_size = shard_size
        self.number_of_possible_windows_per_shard = ((window_size * shard_size - window_size)//stride)+ 1

    def __len__(self):
        return (self.real_nb_windows*self.window_size - self.window_size)//self.stride + 1
    def __getitem__(self , index):
        shard_number = index // self.number_of_possible_windows_per_shard

        

In [13]:
import os
from pathlib import Path
import bisect
import numpy as np
import torch
from torch.utils.data import Dataset
from collections import OrderedDict

class Encoder_Dataset(Dataset):
    def __init__(self, data_dir, window_size=200, stride=20,
                 split="train", shard_size=1000, cache_size=2):
        self.data_dir = Path(data_dir)
        self.window_size = int(window_size)
        self.stride = int(stride)
        self.split = split
        self.shard_size = int(shard_size)
        self.cache_size = cache_size

        split_dir = self.data_dir / split
        if not split_dir.exists():
            raise FileNotFoundError(split_dir)

        # discover shards
        self.shard_paths = sorted(
            (p for p in split_dir.glob("window_shard_*.npy")),
            key=lambda p: int(p.stem.split("_")[-1])
        )
        if not self.shard_paths:
            raise RuntimeError("No shard files matching window_shard_*.npy")

        # per-shard metadata
        self.shard_meta = []
        self.prefix_virtual = [0]
        total_virtual = 0
        for path in self.shard_paths:
            arr = np.load(path, mmap_mode="r")
            n_base, C, W = arr.shape
            if W != self.window_size:
                raise ValueError(f"{path.name}: W={W} != window_size={self.window_size}")
            total_samples = n_base * self.window_size
            n_virtual = 0
            if total_samples >= self.window_size:
                n_virtual = ((total_samples - self.window_size) // self.stride) + 1
            self.shard_meta.append({
                "path": path,
                "n_base": int(n_base),
                "C": int(C),
                "W": int(W),
                "n_virtual": int(n_virtual),
            })
            total_virtual += n_virtual
            self.prefix_virtual.append(total_virtual)
        self.total_virtual = total_virtual
        if self.total_virtual == 0:
            raise RuntimeError("No virtual windows possible.")

        # LRU cache for mmaps (key -> array)

    def __len__(self):
        return self.total_virtual

    def _load_shard(self, shard_idx: int):
        path = self.shard_meta[shard_idx]["path"]
        key = str(path)

        # load fresh
        mm = np.load(path, mmap_mode="r")

        return mm

    def _locate(self, global_index: int):
        s = bisect.bisect_right(self.prefix_virtual, global_index) - 1
        local_idx = global_index - self.prefix_virtual[s]
        return s, local_idx

    def __getitem__(self, index: int):
        if index < 0 or index >= self.total_virtual:
            raise IndexError

        shard_idx, local_idx = self._locate(index)
        meta = self.shard_meta[shard_idx]
        shard = self._load_shard(shard_idx)

        W = self.window_size
        start = local_idx * self.stride
        base_idx = start // W
        offset = start % W

        if offset == 0:
            x = shard[base_idx]
        else:
            first = shard[base_idx, :, offset:]
            need = W - first.shape[1]
            second = shard[base_idx + 1, :, :need]
            x = np.concatenate([first, second], axis=1)

        return torch.from_numpy(np.asarray(x, dtype=np.float32))


In [14]:
output_dir = r"C:\disque d\ai_stuff\projects\pytorchtraining\eeg_competition\full_eeg_data_shards"
train_dataset = Encoder_Dataset(output_dir , split="train")


In [15]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:

for element in tqdm(train_loader):
    pass